In [20]:
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Lambda
from keras.layers import Embedding
from keras.preprocessing.sequence import pad_sequences
from numpy import array
import string

In [21]:
def load_file(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

def process(data):
    words = data.split()
    words = [word.lower() for word in words]
    final = []
    for word in words:
        try:
            val = int(word)
        except ValueError:
            for ch in word:
                final.append(ch)
            final.append(' ')
    return final

In [22]:
shakespeare = load_file('data/shakespeare.txt')
data = process(shakespeare)

In [23]:
length = 40 + 1
sequences = list()
for i in range(length, len(data), 10):
    seq = data[i-length:i]
    sequences.append(seq)

In [24]:
print(sequences[0])
print(len(sequences[0]))
print(sequences[1])
print(len(data))
print(len(sequences))

['f', 'r', 'o', 'm', ' ', 'f', 'a', 'i', 'r', 'e', 's', 't', ' ', 'c', 'r', 'e', 'a', 't', 'u', 'r', 'e', 's', ' ', 'w', 'e', ' ', 'd', 'e', 's', 'i', 'r', 'e', ' ', 'i', 'n', 'c', 'r', 'e', 'a', 's', 'e']
41
['s', 't', ' ', 'c', 'r', 'e', 'a', 't', 'u', 'r', 'e', 's', ' ', 'w', 'e', ' ', 'd', 'e', 's', 'i', 'r', 'e', ' ', 'i', 'n', 'c', 'r', 'e', 'a', 's', 'e', ',', ' ', 't', 'h', 'a', 't', ' ', 't', 'h', 'e']
93674
9364


In [25]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sequences)
sonnet = tokenizer.texts_to_sequences(sequences)
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

38


In [26]:
sonnet = array(sonnet)
print(sonnet[1])
print(len(sonnet[1]))

[ 6  3  1 20 10  2  7  3 13 10  2  6  1 16  2  1 12  2  6  8 10  2  1  8
  9 20 10  2  7  6  2 17  1  3  5  7  3  1  3  5  2]
41


In [27]:
X = sonnet[:, :-1]
y = sonnet[:, -1]
y = to_categorical(y, num_classes=vocab_size)
print(y[1])
seq_length = X.shape[1]
print(seq_length)

[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
40


In [28]:
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=seq_length))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Lambda(lambda x: x / 1.5))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 40, 100)           3800      
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
lambda_2 (Lambda)            (None, 100)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 38)                3838      
Total params: 98,138
Trainable params: 98,138
Non-trainable params: 0
_________________________________________________________________
None


In [29]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, batch_size=128, epochs=100)

Epoch 1/100
9364/9364 [==============================] - 9s 908us/step - loss: 3.1043 - acc: 0.1826

In [30]:
seed_text = "shall i compare thee to a summer's day?\n"
seed_text = seed_text.split()
print(seed_text)
seed = []
for word in seed_text:
    for ch in word:
        seed.append(ch)
    seed.append(' ')
print(seed)

['shall', 'i', 'compare', 'thee', 'to', 'a', "summer's", 'day?']
['s', 'h', 'a', 'l', 'l', ' ', 'i', ' ', 'c', 'o', 'm', 'p', 'a', 'r', 'e', ' ', 't', 'h', 'e', 'e', ' ', 't', 'o', ' ', 'a', ' ', 's', 'u', 'm', 'm', 'e', 'r', "'", 's', ' ', 'd', 'a', 'y', '?', ' ']


In [31]:
encoded = tokenizer.texts_to_sequences([seed])[0]
encoded = np.asarray(encoded)
print(encoded)
print(encoded.shape)
print(model.predict_classes(np.reshape(encoded, (1, 40)), verbose=0))

[ 6  5  7 11 11  1  8  1 20  4 14 22  7 10  2  1  3  5  2  2  1  3  4  1
  7  1  6 13 14 14  2 10 26  6  1 12  7 15 28  1]
(40,)
[16]


In [13]:
print("shall i compare thee to a summer's day?\n")
result = []
for x in range(13):
    temp_line = []
    for y in range(40):
        encoded = tokenizer.texts_to_sequences([seed])[0]
        encoded = pad_sequences([encoded], maxlen=40, truncating='pre')
        yhat = model.predict_classes(np.reshape(encoded, (1, 40)), verbose=0)
        for w, index in tokenizer.word_index.items():
            if index == yhat:
                letter = w
                temp_line.append(letter)
                seed.append(letter)
                break
    result.append(temp_line)
for line in result:
    print(''.join(line))


shall i compare thee to a summer's day?

sclly st not nche, as bllls am of hind a
rker herare inced that i parighasy, whee
 in ne corcooveled that thet no, which o
ral as fires of alles have wast onchece 
thee, saven's, all thy seef in, arad tha
t pyysaal, as frind my selfallt, and all
l il is us wist not so, seefore staut i 
my will negre, whee in most strig cease,
 so with tou my sigllllllll, rigs dot do
t veat be chang: all thous havilak haild
 in dove as praise with thee, frem the f
are is bife of the eettor tom, ever chan
kel, as pruch ulourheref llet the wruce 


In [46]:
print("shall i compare thee to a summer's day?\n")
result = []
for x in range(13):
    temp_line = []
    for y in range(40):
        encoded = tokenizer.texts_to_sequences([seed])[0]
        encoded = pad_sequences([encoded], maxlen=40, truncating='pre')
        yhat = model.predict_classes(np.reshape(encoded, (1, 40)), verbose=0)
        for w, index in tokenizer.word_index.items():
            if index == yhat:
                letter = w
                temp_line.append(letter)
                seed.append(letter)
                break
    result.append(temp_line)
for line in result:
    print(''.join(line))

shall i compare thee to a summer's day?

e i so seef in not sholl swit, for stun 
of net thought love thee thee a gackal c
hive, i peet inked wher plew it vave cun
e, one coon o he dist do decosst, will i
t best pine. hereingh at thou strang pei
terer of would thee art all illivedy whe
re frob hours pive worst, will from my a
fort despire, wherefor's whee art tells 
of as as as as as asose pecetiscce on sc
isting that wong mest witn should mourou
r i manks the striess my verwary, where 
noth that would my should cruep, wither 
and the worllins so reme: bell. and yet 


In [424]:
print("shall i compare thee to a summer's day?\n")
result = []
for x in range(13):
    temp_line = []
    for y in range(40):
        encoded = tokenizer.texts_to_sequences([seed])[0]
        encoded = pad_sequences([encoded], maxlen=40, truncating='pre')
        yhat = model.predict_classes(np.reshape(encoded, (1, 40)), verbose=0)
        for w, index in tokenizer.word_index.items():
            if index == yhat:
                letter = w
                temp_line.append(letter)
                seed.append(letter)
                break
    result.append(temp_line)
for line in result:
    print(''.join(line))

shall i compare thee to a summer's day?

hath in thee thou stave tast not love th
at will day what wif hath the bott,, and
 ter fonne, and thrmbe thath mast dessue
tlf ling, ar bl nececcssioond whorger th
ant that tim thy wart of merfich, stilg,
 and im not of dis i gong with thee mine
 end, suing that you me caccooousolneccc
ousty, bive to thatly wher do dhough the
e mine of reet in gooved that texpere, w
ho gived theref nor shat have thou thost
 will ay clovelody ar thou art and loved
 of thee art begutt our let lettrle: a p
ory sin, and thrmer thath dis me, thou (


In [19]:
print("shall i compare thee to a summer's day?\n")
result = []
for x in range(14):
    temp_line = []
    for y in range(40):
        encoded = tokenizer.texts_to_sequences([seed])[0]
        encoded = pad_sequences([encoded], maxlen=40, truncating='pre')
        yhat = model.predict_classes(np.reshape(encoded, (1, 40)), verbose=0)
        for w, index in tokenizer.word_index.items():
            if index == yhat:
                letter = w
                temp_line.append(letter)
                seed.append(letter)
                break
    result.append(temp_line)
for line in result:
    print(''.join(line))

shall i compare thee to a summer's day?

vere. glood as now least lover's cremme'
s a play, bigiliss andowing this mony, s
efe there abwales wand thou made, aguf f
oo low, to were ind i mance every by pui
ghts, the cont's rebpont hos, migione as
 as fime. when that for sich of hing: o 
thou a tane care, which hast thou the th
at worthlest to to glen blook all sim, y
et whe this offor's rrespec of lose, wan
d i casceason o a shaing: and worneny on
 mow. thought the thous ast and thy fart
ine and that wa post still by mast, you 
on o toug tells asseast by make to ceart
s pressot it. and therefore from there s


In [45]:
print("shall i compare thee to a summer's day?\n")
result = []
for x in range(14):
    temp_line = []
    for y in range(40):
        encoded = tokenizer.texts_to_sequences([seed])[0]
        encoded = pad_sequences([encoded], maxlen=40, truncating='pre')
        yhat = model.predict_classes(np.reshape(encoded, (1, 40)), verbose=0)
        for w, index in tokenizer.word_index.items():
            if index == yhat:
                letter = w
                temp_line.append(letter)
                seed.append(letter)
                break
    result.append(temp_line)
for line in result:
    print(''.join(line))

shall i compare thee to a summer's day?

 where cunt it bet' for her chording wou
ld my deruge all the wast is vaty, for s
hall sinl hearkelle, ar whee stef to a e
vewary ard wast, when i sweet ommery who
u is havks pleasur, whe clest with my de
ars, whe chost self when in my gerarke m
y defbllest prosty stettisge of the fors
s o he thand fromblins prominds tainst t
hat it tnou shall. som sheet't wheef the
e thee thee in my gubligs trate of yow, 
and with the ay at you thy swill nes pre
ce is hagk's pleasure my love, the ind t
ray allowy and will for me touch afory m
y glan brang was pipe, age tesss-tolt se
